## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
import random
from pprint import pprint
from time import time
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode
from multiprocess import Pool
from math import ceil
from dotenv import dotenv_values
from IPython.display import display, HTML

from contract_utils import execute_bash, instantiate_contract, execute_contract_bash, query_contract, get_ipfs_cid_from_str, get_proofs, ContractUtils


ipfs_client = ipfshttpclient.connect()

NUMBER_OF_PARTICIPANTS = 50_000
NUMBER_OF_ACTIVATED_PARTICIPANTS = 35_000
NUMBER_OF_THREADS = 100
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [get_ipfs_cid_from_str(_nickname + '_avatar') for _nickname in NICKNAME_LIST]
CLAIM_AMOUNT_LIST = [30_000] * NUMBER_OF_PARTICIPANTS
PREFIXES = ['cosmos', 'osmo', 'terra']

INITIAL_BALANCE = str(7_000_000_000)
COEF_UP = str(13)
COEF_DOWN = str(7)
TARGET_CLAIM = str(20_000)

WALLET_ADDRESS = dotenv_values('.env')['WALLET_ADDRESS']
WALLET_SEED = dotenv_values('.env')['WALLET_SEED']
PYTHON_PATH = dotenv_values('.env')['PYTHON_PATH']

DISPLAY_TX_EXECUTION = False
CALCULATE_ADDRESS_SET = True
CALCULATE_PROOFS = True
SEND_COINS = True

INIT_SUBGRAPH_CONTRACTS = False
SUBGRAPH_CODE_ID = str(3)
NAME_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1qg5ega6dykkxc307y25pecuufrjkxkaggkkxh7nad0vhyhtuhw3sp98tza'
AVATAR_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom17p9rzwnnfxcjp32un9ug7yhhzgtkhvl9jfksztgw5uh69wac2pgs6qwudf'
PROOF_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1unyuj8qnmygvzuex3dwmg9yzt9alhvyeat0uu0jedg2wj33efl5qpj0rgp'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(2)
PASSPORT_CONTRACT_ADDRESS = 'bostrom1hulx7cgvpfcvg83wk5h96sedqgn72n026w6nl47uht554xhvj9nsjxcwgf'

INIT_TREASURY_CONTRACT = False
TREASURY_CODE_ID = str(4)
TREASURY_CONTRACT_ADDRESS = 'bostrom18cszlvm6pze0x9sz32qnjq4vtd45xehqs8dq7cwy8yhq35wfnn3q9u647v'

INIT_GIFT_CONTRACT = True
GIFT_CODE_ID = str(1)
GIFT_CONTRACT_ADDRESS = 'bostrom14swjuycwxmvghd9hgr3f3755388g6xkaqpn2dxcf700fzcwdm2nqum7453'

PARTICIPANTS_FILE_NAME = 'participants_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv'
ROOT_SOURCE_FILE_NAME = 'root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
PROOF_FILE_NAME = 'proof_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'

/home/grovybear/.local/lib/python3.9/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.12.2' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Generate addresses and sign messages

In [2]:
def generate_wallet_data(i: int, prefixes: list = PREFIXES) -> dict:

    prefix = random.choice(prefixes)

    claim_item = {'amount': CLAIM_AMOUNT_LIST[i],
              'nickname': NICKNAME_LIST[i],
              'avatar': AVATAR_CID_LIST[i]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], prefix)
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    claim_item['message'] = f"{claim_item['bostrom_address']}:QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV"

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix=prefix)
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    return claim_item


if CALCULATE_ADDRESS_SET:

    tasks = list(range(NUMBER_OF_PARTICIPANTS))
    print(f'Number of tasks: {len(tasks)}')
    print(f'Number of threads: {NUMBER_OF_THREADS}')
    with Pool(processes=NUMBER_OF_THREADS) as pool:
        claims_list = list(tqdm(pool.imap(generate_wallet_data, tasks), total=len(tasks)))

    claims_df = pd.DataFrame(claims_list)
    claims_df.to_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
else:
    claims_df = pd.read_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df.head()

Number of tasks: 50000
Number of threads: 100


  0%|          | 0/50000 [00:00<?, ?it/s]

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,30000,john16510755980,QmT3nshpKe2jDXsqYbWseoKLqNk5hkfSNkXfbegULUSWTe,0x4728b5d6240343f357b7e11863ad32492ebc7668,0x6dca8d2894e3e4fcbb126639aba5334e5bc46ce9c50e...,bostrom12al99ea87hnxq52jc577ulzmkn63vhzhtglj70,terra12al99ea87hnxq52jc577ulzmkn63vhzhwl3pzg,enjoy wait anxiety recycle thrive evidence was...,bostrom12al99ea87hnxq52jc577ulzmkn63vhzhtglj70...,0xed64ce431d3729e95fe6cd7fd4c735a609807602cedf...,{'pub_key': 'A+/lC0Nmk1Fbl2ImpY4Gsli81L4isAJYE...,eyJwdWJfa2V5IjoiQSsvbEMwTm1rMUZibDJJbXBZNEdzbG...
1,30000,john16510755981,QmZpd2kCy2L32nutybeNyZa2U4V8MEVfG9t3xsHV8hi8oi,0x8544acb35c255078dcfe5827bdf539f23fc4f347,0xd08ff213baeee47add74b3620332e595f8edb5dc556f...,bostrom1en9pu3xckqu8cj6qenw9x0xs02zfe8h9eah03s,cosmos1en9pu3xckqu8cj6qenw9x0xs02zfe8h96wru0h,dry auto denial present oval marine opinion sl...,bostrom1en9pu3xckqu8cj6qenw9x0xs02zfe8h9eah03s...,0x9225d7843f8bcdf7c4e969a69cba1ab9bb2e2f83ede8...,{'pub_key': 'Aj7QPDdHomIP2DUH9eTnSfOon44xrMLIZ...,eyJwdWJfa2V5IjoiQWo3UVBEZEhvbUlQMkRVSDllVG5TZk...
2,30000,john16510755982,QmaSExj3kqGgyNHDcS1MYi4RRWRRhiLTMQvJBVSV1pRysv,0x9d265a500b932d1601478d50af8fa597b51077ed,0x5d7d660a54591c323a836e162a9865a245b738c2d548...,bostrom1a2z98w5htp738l35vra8lcknztf292mysr7spz,cosmos1a2z98w5htp738l35vra8lcknztf292myns2rl9,ship turn amused rule parrot current hold regr...,bostrom1a2z98w5htp738l35vra8lcknztf292mysr7spz...,0xd4000eacac3facfc3366bc893bfdd7a0ea24288650e7...,{'pub_key': 'Aw0oWw3gkVlNat5+055ePbrJhrj7LVZy0...,eyJwdWJfa2V5IjoiQXcwb1d3M2drVmxOYXQ1KzA1NWVQYn...
3,30000,john16510755983,QmdkNrH37aHtjkqc34wWfNr69UvQXrsghmpRDb8N62RBcK,0x0ccdb3913071a85ba7e325a728b35b7031a2f841,0xe65d014bde708de4d4558e7c09b45b08e1d1f0f04fb5...,bostrom1a00xqte46sv80e386jxjen7htpzuk64s9lmqmn,terra1a00xqte46sv80e386jxjen7htpzuk64sqg4n85,side bread decline actor mandate hollow arena ...,bostrom1a00xqte46sv80e386jxjen7htpzuk64s9lmqmn...,0x159ee0c77d0a03074fd7fe418ec929d28d9919ca44ed...,{'pub_key': 'Aw8kHHa+WGrKrX0aS3j4V3RP+Gzusj1b/...,eyJwdWJfa2V5IjoiQXc4a0hIYStXR3JLclgwYVMzajRWM1...
4,30000,john16510755984,QmWDPSpPxJdVkefCQT7CDP1dGWMkcPdGUavCR9JBTZEdcu,0xe31aa9aee81531fcc8b837653a4b565607fe3793,0xcc56f1eebcea455d20304a178b8d96cba6aba371bbfd...,bostrom144x8pazv6ueln6wyv4ze6aq6q528mu0vnuy7pt,terra144x8pazv6ueln6wyv4ze6aq6q528mu0vkt2dav,text rival knock hire feature cabbage toilet v...,bostrom144x8pazv6ueln6wyv4ze6aq6q528mu0vnuy7pt...,0x286d0513c0301039babb77e31d091b8a7680917943aa...,{'pub_key': 'AhGcw/spF88f7kZgMhHeFMRFE83QX5iD8...,eyJwdWJfa2V5IjoiQWhHY3cvc3BGODhmN2taZ01oSGVGTV...


### Create Merkle Tree, Get Root and Proofs

In [3]:
root_source_list = [{'address': _item[0],
                     'amount': str(_item[1])} for _item in claims_df[['ethereum_address', 'amount']].values.tolist()]+ \
                    [{'address': _item[0],
                      'amount': str(_item[1])} for _item in claims_df[['cosmos_address', 'amount']].values.tolist()]
root_source_list.append({'address': '0xF2749114FeaAD68854E01C8eE762C7170532FdfD'.lower(), 'amount': '10000000'})
with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [4]:
NUMBER_OF_THREADS = 15
BASH_SIZE = 1000

tasks = set(
    (
        (ROOT_SOURCE_FILE_NAME,
         f'temp/proofs_{i}.json',
         i * BASH_SIZE,
         min(NUMBER_OF_PARTICIPANTS * 2 + 1, (i + 1) * BASH_SIZE + 1))
        for i in range(ceil(2 * NUMBER_OF_PARTICIPANTS/BASH_SIZE))
    )
)
if CALCULATE_PROOFS:
    print(f'Number of addresses: {2 * NUMBER_OF_PARTICIPANTS}\nNumber of threads: {NUMBER_OF_THREADS}\nBash size: {BASH_SIZE}\nNumber of tasks: {tasks}')
    with Pool(processes=NUMBER_OF_THREADS) as pool:
        res = pool.starmap(get_proofs, tqdm(tasks, total=len(tasks)))
    assert res == [True] * len(res)

{('root_testing_source_50000_addresses.json', 'temp/proofs_75.json', 75000, 76001), ('root_testing_source_50000_addresses.json', 'temp/proofs_8.json', 8000, 9001), ('root_testing_source_50000_addresses.json', 'temp/proofs_5.json', 5000, 6001), ('root_testing_source_50000_addresses.json', 'temp/proofs_37.json', 37000, 38001), ('root_testing_source_50000_addresses.json', 'temp/proofs_4.json', 4000, 5001), ('root_testing_source_50000_addresses.json', 'temp/proofs_17.json', 17000, 18001), ('root_testing_source_50000_addresses.json', 'temp/proofs_19.json', 19000, 20001), ('root_testing_source_50000_addresses.json', 'temp/proofs_82.json', 82000, 83001), ('root_testing_source_50000_addresses.json', 'temp/proofs_48.json', 48000, 49001), ('root_testing_source_50000_addresses.json', 'temp/proofs_95.json', 95000, 96001), ('root_testing_source_50000_addresses.json', 'temp/proofs_81.json', 81000, 82001), ('root_testing_source_50000_addresses.json', 'temp/proofs_23.json', 23000, 24001), ('root_testi

  0%|          | 0/100 [00:00<?, ?it/s]

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_50000_addresses.json --output temp/proofs_31.json --start_index 31000 --end_index 32001
Merkle root: a29db2dd9b82645fec8fe623533e7c1ede6c8a9cd549ff3d27d986d77c7358b4
Number of addresses in the Merkle tree: 1001
Done in 20.86s.

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_50000_addresses.json --output temp/proofs_2.json --start_index 2000 --end_index 3001
Merkle root: a29db2dd9b82645fec8fe623533e7c1ede6c8a9cd549ff3d27d986d77c7358b4
Number of addresses in the Merkle tree: 1001
Done in 22.00s.

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_50000_addresses.json --output temp/proofs_30.json --start_index 30000 --end_index 31001
Merkle root: a29db2dd9b82645fec8fe623533e7c1ede6c8a9cd549ff3d27d986d77c7358b4
Number of addresses in the Merkle tree: 1001
Done in 23.40s.

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_50000_addresses.json --output temp/proofs_5.json --start_in

In [5]:
if CALCULATE_PROOFS:
    roots = []
    proofs_df = pd.DataFrame(columns=['address', 'amount', 'proof'])
    for task in tasks:
        with open(task[1], 'r') as proof_file:
            root_and_proof_json = json.load(proof_file)
        roots.append(root_and_proof_json['merkle_root'])
        proofs_df = pd.concat([proofs_df, pd.DataFrame(root_and_proof_json['proofs'])], ignore_index=True)
    assert roots == [roots[0]] * len(roots)
    root = roots[0]

    cosmos_proofs_df = proofs_df[~proofs_df.address.str.startswith('0x')][['address', 'proof']]
    ethereum_proofs_df = proofs_df[proofs_df.address.str.startswith('0x')][['address', 'proof']]

    claims_with_proofs_df = claims_df\
        .merge(
            ethereum_proofs_df.rename(columns={'address': 'ethereum_address', 'proof': 'ethereum_proof'}),
            how='left',
            on='ethereum_address')\
        .merge(
            cosmos_proofs_df.rename(columns={'address': 'cosmos_address', 'proof': 'cosmos_proof'}),
            how='left',
            on='cosmos_address')

    claims_with_proofs_df.to_csv(PARTICIPANTS_FILE_NAME)
else:
    claims_with_proofs_df = pd.read_csv(PARTICIPANTS_FILE_NAME)
    claims_with_proofs_df.loc[:, 'ethereum_proof'] = claims_with_proofs_df['ethereum_proof'].map(lambda x: x.replace('\'', '').replace('[', '').replace(']', '').split(', '))
    claims_with_proofs_df.loc[:, 'cosmos_proof'] = claims_with_proofs_df['cosmos_proof'].map(lambda x: x.replace('\'', '').replace('[', '').replace(']', '').split(', '))
    with open('temp/proofs_1.json', 'r') as proof_file:
        root_and_proof_json = json.load(proof_file)
    root = root_and_proof_json['merkle_root']
claims_with_proofs_df.head()

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature,ethereum_proof,cosmos_proof
0,30000,john16510755980,QmT3nshpKe2jDXsqYbWseoKLqNk5hkfSNkXfbegULUSWTe,0x4728b5d6240343f357b7e11863ad32492ebc7668,0x6dca8d2894e3e4fcbb126639aba5334e5bc46ce9c50e...,bostrom12al99ea87hnxq52jc577ulzmkn63vhzhtglj70,terra12al99ea87hnxq52jc577ulzmkn63vhzhwl3pzg,enjoy wait anxiety recycle thrive evidence was...,bostrom12al99ea87hnxq52jc577ulzmkn63vhzhtglj70...,0xed64ce431d3729e95fe6cd7fd4c735a609807602cedf...,{'pub_key': 'A+/lC0Nmk1Fbl2ImpY4Gsli81L4isAJYE...,eyJwdWJfa2V5IjoiQSsvbEMwTm1rMUZibDJJbXBZNEdzbG...,[34b7a2e200e5c3f096bdc387bba1de12394b5bf22fbc6...,[0c9f9a62d402bb94f072f4e0d3315facacdde304c170f...
1,30000,john16510755980,QmT3nshpKe2jDXsqYbWseoKLqNk5hkfSNkXfbegULUSWTe,0x4728b5d6240343f357b7e11863ad32492ebc7668,0x6dca8d2894e3e4fcbb126639aba5334e5bc46ce9c50e...,bostrom12al99ea87hnxq52jc577ulzmkn63vhzhtglj70,terra12al99ea87hnxq52jc577ulzmkn63vhzhwl3pzg,enjoy wait anxiety recycle thrive evidence was...,bostrom12al99ea87hnxq52jc577ulzmkn63vhzhtglj70...,0xed64ce431d3729e95fe6cd7fd4c735a609807602cedf...,{'pub_key': 'A+/lC0Nmk1Fbl2ImpY4Gsli81L4isAJYE...,eyJwdWJfa2V5IjoiQSsvbEMwTm1rMUZibDJJbXBZNEdzbG...,[34b7a2e200e5c3f096bdc387bba1de12394b5bf22fbc6...,[0c9f9a62d402bb94f072f4e0d3315facacdde304c170f...
2,30000,john16510755981,QmZpd2kCy2L32nutybeNyZa2U4V8MEVfG9t3xsHV8hi8oi,0x8544acb35c255078dcfe5827bdf539f23fc4f347,0xd08ff213baeee47add74b3620332e595f8edb5dc556f...,bostrom1en9pu3xckqu8cj6qenw9x0xs02zfe8h9eah03s,cosmos1en9pu3xckqu8cj6qenw9x0xs02zfe8h96wru0h,dry auto denial present oval marine opinion sl...,bostrom1en9pu3xckqu8cj6qenw9x0xs02zfe8h9eah03s...,0x9225d7843f8bcdf7c4e969a69cba1ab9bb2e2f83ede8...,{'pub_key': 'Aj7QPDdHomIP2DUH9eTnSfOon44xrMLIZ...,eyJwdWJfa2V5IjoiQWo3UVBEZEhvbUlQMkRVSDllVG5TZk...,[30f68c9d61928bcedeadb3b832f7a05564acf39862de9...,[4327d418680f19edea0c09eb64a6856da55bee3733390...
3,30000,john16510755982,QmaSExj3kqGgyNHDcS1MYi4RRWRRhiLTMQvJBVSV1pRysv,0x9d265a500b932d1601478d50af8fa597b51077ed,0x5d7d660a54591c323a836e162a9865a245b738c2d548...,bostrom1a2z98w5htp738l35vra8lcknztf292mysr7spz,cosmos1a2z98w5htp738l35vra8lcknztf292myns2rl9,ship turn amused rule parrot current hold regr...,bostrom1a2z98w5htp738l35vra8lcknztf292mysr7spz...,0xd4000eacac3facfc3366bc893bfdd7a0ea24288650e7...,{'pub_key': 'Aw0oWw3gkVlNat5+055ePbrJhrj7LVZy0...,eyJwdWJfa2V5IjoiQXcwb1d3M2drVmxOYXQ1KzA1NWVQYn...,[288660d1d333ebb6696458bfa55d84ef1ef3d72ca60f3...,[fd4f92e4df636bf36fa59bd8e58cb73db8f047c390fe8...
4,30000,john16510755983,QmdkNrH37aHtjkqc34wWfNr69UvQXrsghmpRDb8N62RBcK,0x0ccdb3913071a85ba7e325a728b35b7031a2f841,0xe65d014bde708de4d4558e7c09b45b08e1d1f0f04fb5...,bostrom1a00xqte46sv80e386jxjen7htpzuk64s9lmqmn,terra1a00xqte46sv80e386jxjen7htpzuk64sqg4n85,side bread decline actor mandate hollow arena ...,bostrom1a00xqte46sv80e386jxjen7htpzuk64s9lmqmn...,0x159ee0c77d0a03074fd7fe418ec929d28d9919ca44ed...,{'pub_key': 'Aw8kHHa+WGrKrX0aS3j4V3RP+Gzusj1b/...,eyJwdWJfa2V5IjoiQXc4a0hIYStXR3JLclgwYVMzajRWM1...,[d1e1ef301abdeba9df2dd907e330b33160c8cb3e4925c...,[176519937326fb3e1316b4beb5814163e26fdc9b864f7...


In [6]:
for i in range(ceil(NUMBER_OF_PARTICIPANTS/10_000)):
    claims_with_proofs_df.loc[i*10_000:(i+1)*10_000,:].to_csv(f'temp/{i}_{PARTICIPANTS_FILE_NAME}')

## Instantiate Contracts
### Instantiate SUBGRAPH Contracts

In [7]:
if INIT_SUBGRAPH_CONTRACTS:
    name_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test name subgraph')
    avatar_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test avatar subgraph',
            from_address=WALLET_ADDRESS)
    proof_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test proof subgraph',
            from_address=WALLET_ADDRESS)
else:
    name_subgraph_contract_address = NAME_SUBGRAPH_CONTRACT_ADDRESS
    avatar_subgraph_contract_address = AVATAR_SUBGRAPH_CONTRACT_ADDRESS
    proof_subgraph_contract_address = PROOF_SUBGRAPH_CONTRACT_ADDRESS
print(f'Name subgraph contract address: {name_subgraph_contract_address}\n'
      f'Avatar subgraph contract address: {avatar_subgraph_contract_address}\n'
      f'Proof subgraph contract address: {proof_subgraph_contract_address}')

Name subgraph contract address: bostrom1qg5ega6dykkxc307y25pecuufrjkxkaggkkxh7nad0vhyhtuhw3sp98tza
Avatar subgraph contract address: bostrom17p9rzwnnfxcjp32un9ug7yhhzgtkhvl9jfksztgw5uh69wac2pgs6qwudf
Proof subgraph contract address: bostrom1unyuj8qnmygvzuex3dwmg9yzt9alhvyeat0uu0jedg2wj33efl5qpj0rgp


### Instantiate Passport Contract

In [8]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subgraph": "{avatar_subgraph_contract_address}", "name_subgraph": "{name_subgraph_contract_address}", "proof_subgraph": "{proof_subgraph_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport',
            from_address=WALLET_ADDRESS)
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom1hulx7cgvpfcvg83wk5h96sedqgn72n026w6nl47uht554xhvj9nsjxcwgf


### Set executor in the Subgraph Contracts

In [9]:
def set_executor_subgraph(subgraph_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_contract_bash(execute_query=f'''{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}''',
                                 contract_address=subgraph_contract_address,
                                 gas=600000,
                                 display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBGRAPH_CONTRACTS:
    set_executor_subgraph(subgraph_contract_address=name_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=avatar_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=proof_subgraph_contract_address, new_executor_address=passport_contract_address)

### Instantiate Treasury Contract

In [60]:
if INIT_TREASURY_CONTRACT:
    treasury_contract_address = \
        instantiate_contract(
            init_query=f'''{{"admins": ["{WALLET_ADDRESS}"], "mutable": true}}''',
            contract_code_id=TREASURY_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test treasury',
            from_address=WALLET_ADDRESS,
            display_data=False)
else:
    treasury_contract_address = TREASURY_CONTRACT_ADDRESS
print(f'Treasury contract address: {treasury_contract_address}')

Treasury contract address: bostrom18cszlvm6pze0x9sz32qnjq4vtd45xehqs8dq7cwy8yhq35wfnn3q9u647v


### Instantiate Gift Contract

In [11]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "treasury":"{treasury_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            contract_label='test gift',
            from_address=WALLET_ADDRESS,
            display_data=False)
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom1kwdranvwf6vl2grh99layugwdnph6um2x0c0l8qyvrcgsjcykuhsjvtnl3


### Initiate Class of Output Parsing

In [12]:
contract_utils = ContractUtils(ipfs_client=ipfs_client,
                               address_dict= {
                                   gift_contract_address: 'Gift Contract',
                                   passport_contract_address: 'Passport Contract',
                                   treasury_contract_address: 'Treasury Contract',
                                   WALLET_ADDRESS: 'Passport Owner Address',
                                   name_subgraph_contract_address: 'Name Subgraph Contract',
                                   avatar_subgraph_contract_address: 'Avatar Subgraph Contract',
                                   proof_subgraph_contract_address: 'Proof Subgraph Contract'})

### Set Allowance in Treasury contract

In [13]:
contract_utils.parse_contract_execution_json(
    contract_utils.send_coins(
        from_seed=WALLET_SEED,
        to_addresses=[treasury_contract_address],
        amounts=[INITIAL_BALANCE],
        gas=min(100_000 + 25_000, int(20e6)),
        display_data=DISPLAY_TX_EXECUTION))


Events

coin received
	receiver: Treasury Contract
	amount: 7000000000boot

coin spent
	spender: Passport Owner Address
	amount: 7000000000boot

message from bank bostrom1mxdtr8lruutugqtxgpw2sf2tl2mhzlq5fd2du0 /cosmos.bank.v1beta1.MsgMultiSend

transfer
	recipient: Treasury Contract
	amount: 7000000000boot
Gas used: 72,935
Tx hash: DB7D682C80C98487226020809543360F870D4334CA1A4E81092AD316A6E9D7E9


In [54]:
def set_allowance(amount: str = INITIAL_BALANCE, display_data: bool = False):
    return contract_utils.execute_contract(
        execute_msg={"increase_allowance": {"spender": gift_contract_address, "amount": {"amount": amount, "denom": 'boot'}}},
        contract_address=treasury_contract_address,
        mnemonic=WALLET_SEED,
        gas=600000,
        display_data=display_data)

contract_utils.parse_contract_execution_json(
    set_allowance(display_data=False))


Events

execute
	execute contract: Treasury Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Treasury Contract
	action: increase_allowance
	owner: Passport Owner Address
	spender: Gift Contract
	denomination: boot
	amount: 7,000,000,000
Gas used: 135,616
Tx hash: 340EA372979FFB714A5ECD218685EC42EB8F5A81F19FD91C0590E8F12F295ED0


### Register Merkle Root

In [15]:
root_register_output = contract_utils.execute_contract(
    execute_msg={"register_merkle_root": {"merkle_root": root}},
    mnemonic=WALLET_SEED,
    contract_address=gift_contract_address)
contract_utils.parse_contract_execution_json(root_register_output)


Events

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Gift Contract
	action: register_merkle_root
	merkle_root: a29db2dd9b82645fec8fe623533e7c1ede6c8a9cd549ff3d27d986d77c7358b4
Gas used: 130,997
Tx hash: 89BA6F6CB162C54FFA8CBC7E1260520C935DD91A5C285F3F064A4120551907AA


#### Get Merkle Root form the Gift Contract

In [16]:
print(f'Gift contract {gift_contract_address}')
print(query_contract(query='''{"merkle_root": {}}''',
                     contract_address=gift_contract_address))

Gift contract bostrom1kwdranvwf6vl2grh99layugwdnph6um2x0c0l8qyvrcgsjcykuhsjvtnl3
{'data': {'merkle_root': 'a29db2dd9b82645fec8fe623533e7c1ede6c8a9cd549ff3d27d986d77c7358b4'}}


### Send coins to new addresses

In [17]:
NUMBER_ADDRESSES_IN_SENDING_CHUNK = 750

bostrom_addresses = claims_with_proofs_df.bostrom_address.to_list()
bostrom_addresses_chunks = [bostrom_addresses[i: i+ NUMBER_ADDRESSES_IN_SENDING_CHUNK] for i in range(0, len(bostrom_addresses), NUMBER_ADDRESSES_IN_SENDING_CHUNK)]

for bostrom_addresses_item in tqdm(bostrom_addresses_chunks):
    send_output = contract_utils.send_coins(
            from_seed=WALLET_SEED,
            to_addresses=bostrom_addresses_item,
            amounts=[1] * len(bostrom_addresses_item),
            gas=min(150_000 + 30_000 * len(bostrom_addresses_item), int(24e6)),
            display_data=DISPLAY_TX_EXECUTION)


  0%|          | 0/101 [00:00<?, ?it/s]

In [18]:
contract_utils.parse_contract_execution_json(contract_execution_json=send_output)


Events

coin received
	receiver: bostrom1xpr3pj8td0erzen6gh7z7sng98c22mxasxxprn
	amount: 1boot
	receiver: bostrom1z484h86tkakyeq9ws2qanwvmw06yku977gxqkt
	amount: 1boot
	receiver: bostrom1tv67xfv59fysfxuufvqau654qef5uqa33yf4ux
	amount: 1boot
	receiver: bostrom1ewzgdnhgm9zm56vk46gpf9l25h27df6phe7n8e
	amount: 1boot
	receiver: bostrom1v4weesh6z7rg3yx9w66fq2g9vg5cwqdj2m0w2p
	amount: 1boot
	receiver: bostrom1ngf69e5r9uqjtwlqnufh4ddenztqvql79llkn6
	amount: 1boot
	receiver: bostrom1zfsgguyjcwt9r6svf845gpnmcl9hg5g2gy6r3v
	amount: 1boot
	receiver: bostrom1cv2vgwzd2e5hfa2qvdkamg96qajyd9et09gluq
	amount: 1boot
	receiver: bostrom1wa5l578qzpqyjlx9trxhapj2hwhuk4873we4aj
	amount: 1boot
	receiver: bostrom1y89l5jhu2m9j0zkckux00awsuvs46tuyyl3lrx
	amount: 1boot
	receiver: bostrom1nnw8zl79m5cum2eej7z7tqz0u0q7k45yn3vxhh
	amount: 1boot
	receiver: bostrom17l7mv85ajvj84hu5xejtmmqauqrm4t7s4m9xjd
	amount: 1boot
	receiver: bostrom1wqrsxnertyeft5m6et3np7emctqfkjssqsug2s
	amount: 1boot
	receiver: bostrom1dh52nevsd

### Create Passports, Proof Addresses and Claim Gift

In [19]:
def execute_participation(row_index: int) -> bool:
    _output, _error = execute_bash(bash_command=f'{PYTHON_PATH} create_passport_and_claim_job.py temp/{row_index//10_000}_{PARTICIPANTS_FILE_NAME} {row_index} {gift_contract_address}')
    if _output:
        return _output
    else:
        return _error

NUMBER_OF_THREADS = 100

tasks = list(range(NUMBER_OF_ACTIVATED_PARTICIPANTS))
print(f'Number of tasks: {len(tasks)}')
print(f'Number of threads: {NUMBER_OF_THREADS}')


with Pool(processes=NUMBER_OF_THREADS) as pool:
    res_participation = list(tqdm(pool.imap(execute_participation, tasks), total=len(tasks)))

Number of tasks: 35000
Number of threads: 100


  0%|          | 0/35000 [00:00<?, ?it/s]

In [61]:
with open('temp/contract_participation_execution_log_900.txt', 'r') as f:
    data = json.load(f)

In [62]:
contract_utils.parse_contract_execution_json(data['create'], row=claims_with_proofs_df.loc[1,:])


Events

execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0cccqgq wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Name Subgraph Contract
	reply contract: Avatar Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: mint
	minter: Passport Contract
	owner: bostrom1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0cccqgq
	token_id: 1313
Gas used: 304,936
Tx hash: 68476B519B9D3917F8642F5AC5A1072F6920709D4B94B7268695AFA5BE3000F5


In [63]:
contract_utils.parse_contract_execution_json(data['proof_ethereum'], row=claims_with_proofs_df.loc[1,:])


Events

execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0cccqgq wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Proof Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john1651075598899
	address: 0x36b3f2647f2c4fb9a853c84004cf135d1817d3e1
Gas used: 218,815
Tx hash: 941A55ACD76F9448C35805DDF850F091B35F0F65A4FFB22F106845988EF596EA


In [64]:
contract_utils.parse_contract_execution_json(data['proof_cosmos'], row=claims_with_proofs_df.loc[1,:])


Events

execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0cccqgq wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Proof Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john1651075598899
	address: osmo1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0nslrq4
Gas used: 222,147
Tx hash: 1EB352774FC27C7DEB00562B13A2F9EB9AD00B41A0357A9975CDA51B14FF3BA0


In [65]:
contract_utils.parse_contract_execution_json(data['claim_cosmos'], row=claims_with_proofs_df.loc[1,:])


Events

coin received
	receiver: bostrom1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0cccqgq
	amount: 100000boot

coin spent
	spender: Treasury Contract
	amount: 100000boot

execute
	execute contract: Gift Contract
	execute contract: Treasury Contract

message from bostrom1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0cccqgq wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0cccqgq
	sender: Treasury Contract
	amount: 100000boot

wasm
	_contract_address: Gift Contract
	action: claim
	original: osmo1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0nslrq4
	target: bostrom1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0cccqgq
	amount: 372,091
	_contract_address: Treasury Contract
	action: execute
	owner: Gift Contract
Gas used: 394,925
Tx hash: 792776D0D16ED7AB26123A987F98A5D462978CAEBA8D7F77B54BAB2C35B10566


### Release Gift

In [27]:
def execute_release(row_index: int) -> bool:
    _output, _error = execute_bash(bash_command=f'{PYTHON_PATH} create_passport_and_claim_job.py temp/{row_index//10_000}_{PARTICIPANTS_FILE_NAME} {row_index} {gift_contract_address} True')
    if _output:
        return _output
    else:
        return _error


NUMBER_OF_THREADS = 100

tasks = list(range(NUMBER_OF_ACTIVATED_PARTICIPANTS))
print(f'Number of tasks: {len(tasks)}')
print(f'Number of threads: {NUMBER_OF_THREADS}')


with Pool(processes=NUMBER_OF_THREADS) as pool:
    res_release = list(tqdm(pool.imap(execute_release, tasks), total=len(tasks)))

Number of tasks: 35000
Number of threads: 100


  0%|          | 0/35000 [00:00<?, ?it/s]

In [66]:
with open('temp/contract_release_execution_log_900.txt', 'r') as f:
    release_data = json.load(f)

In [67]:
contract_utils.parse_contract_execution_json(release_data['release_ethereum'], row=claims_with_proofs_df.loc[1,:])


Events

coin received
	receiver: bostrom1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0cccqgq
	amount: 27250boot

coin spent
	spender: Treasury Contract
	amount: 27250boot

execute
	execute contract: Gift Contract
	execute contract: Treasury Contract

message from bostrom1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0cccqgq wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0cccqgq
	sender: Treasury Contract
	amount: 27250boot

wasm
	_contract_address: Gift Contract
	action: release
	address: bostrom1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0cccqgq
	gift_address: 0x36b3f2647f2c4fb9a853c84004cf135d1817d3e1
	stage: 9
	amount: 27,250
	_contract_address: Treasury Contract
	action: execute
	owner: Gift Contract
Gas used: 230,828
Tx hash: 41B4CFB4AEACE58CDF5094BC9A64B68CC6EDEDBEFFD2E93BD46274E4C4860067


In [68]:
contract_utils.parse_contract_execution_json(release_data['release_cosmos'], row=claims_with_proofs_df.loc[1,:])


Events

coin received
	receiver: bostrom1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0cccqgq
	amount: 27209boot

coin spent
	spender: Treasury Contract
	amount: 27209boot

execute
	execute contract: Gift Contract
	execute contract: Treasury Contract

message from bostrom1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0cccqgq wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0cccqgq
	sender: Treasury Contract
	amount: 27209boot

wasm
	_contract_address: Gift Contract
	action: release
	address: bostrom1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0cccqgq
	gift_address: osmo1dn752gjrly9y6nm0ddwhd5h3rdp4uwx0nslrq4
	stage: 9
	amount: 27,209
	_contract_address: Treasury Contract
	action: execute
	owner: Gift Contract
Gas used: 230,886
Tx hash: 029259256593E7DD80E682991DE93DB0D9B362FD393BD82931BCFF5800391BA6


## Passport NFT testing
### Transfer Passport

In [31]:
def get_passport_id(bostrom_address: str) -> str:
    return query_contract(query=f'''{{"tokens": {{"owner": "{bostrom_address}"}}}}''',
                          contract_address=passport_contract_address)['data']['tokens'][0]

In [32]:
for index, row in tqdm(claims_with_proofs_df[:4].iterrows()):
    if index == 1:
        transfer_passport_json = contract_utils.transfer_passport(
            row,
            token_id=get_passport_id(row['bostrom_address']),
            to_address=WALLET_ADDRESS,
            display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(
            transfer_passport_json,
            row=row)
    else:
        pass

0it [00:00, ?it/s]


Events

execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom12al99ea87hnxq52jc577ulzmkn63vhzhtglj70 wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Name Subgraph Contract
	reply contract: Avatar Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: transfer_nft
	sender: bostrom12al99ea87hnxq52jc577ulzmkn63vhzhtglj70
	recipient: Passport Owner Address
	token_id: 448
Gas used: 306,534
Tx hash: D8A57F98E6CBF87802C8DEBFB0F9F7A5CFADF987E92EF84A99360F3C88E3BE03


### Burn

In [33]:
for index, row in tqdm(claims_with_proofs_df[2:6].iterrows()):
    if index == 2:
        burn_passport_json = contract_utils.burn_passport(row, token_id=get_passport_id(row['bostrom_address']), display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(burn_passport_json, row=row)
    else:
        pass

0it [00:00, ?it/s]


Events

execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1en9pu3xckqu8cj6qenw9x0xs02zfe8h9eah03s wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Name Subgraph Contract
	reply contract: Avatar Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: burn
	sender: bostrom1en9pu3xckqu8cj6qenw9x0xs02zfe8h9eah03s
	token_id: 419
Gas used: 275,576
Tx hash: F972C65EFB8E00FBCD3AEEE6FC62FCD49CE398EC362A0B573A5989FC0C3F639A


### Add tests for Passport contract
#### create_passport +
#### update_name
#### update_avatar
#### proof_address +
#### remove_address
#### set_minter
#### set_owner
#### set_active
#### set_subgraphs
#### transfer_nft +
#### send_nft
#### mint
#### burn +
#### approve
#### approve_all
#### revoke
#### revoke_all
#### Expirations
##### at_height
##### at_time
##### never

### Add tests for Gift contract
#### update_owner
#### update_passport_addr
#### update_target
#### register_merkle_root +
#### claim +
#### release +

### Add tests for subgraph contract
#### update_owner
#### update_executer +
